# Import

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import xgboost as xgb
from scipy.spatial.distance import squareform, pdist
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 데이터 전처리

## ID별 일일판매량 전처리

In [ ]:
train = pd.read_csv("./train.csv") 

# ['제품', '쇼핑몰'] 기준으로 groupby
grouped_data1 = train.groupby(['제품','쇼핑몰'])
grouped_dfs1 = [group_df for _, group_df in grouped_data1]

# 동일한 쇼핑몰에서 판매하는 동일 제품들의 group index 찾음.
list1 = []
for i in tqdm(range(28863)):
    if len(grouped_dfs1[i]) != 1:
        list1.append(i)

# 동일한 쇼핑몰에서 판매하는 동일 제품들의 'ID'들끼리 일일판매량을 합친 후, 해당 'ID'의 일일판매량으로 대체함.
for i in tqdm(list1):
    a = pd.DataFrame(np.array(grouped_dfs1[i].iloc[0,7:]) + np.array(grouped_dfs1[i].iloc[1,7:]))
    k1 = pd.concat([grouped_dfs1[i].iloc[0,:7].reset_index(drop=True),a],axis=0,ignore_index=True)
    k2 = pd.concat([grouped_dfs1[i].iloc[1,:7].reset_index(drop=True),a],axis=0,ignore_index=True)
    train.loc[train['ID']==k1.loc[0].tolist()[0]] = list(np.array(k1).reshape(-1,486))
    train.loc[train['ID']==k2.loc[0].tolist()[0]] = list(np.array(k2).reshape(-1,486))

train.to_csv("./train1.csv",index=False)

# 쇼핑몰 4와 10에서 판매한 공통 제품들의 일일판매량을 합친 후, 해당 '제품'의 일일판매량으로 대체함. 
shop4 = train[train['쇼핑몰']=='S001-00004']
shop10 = train[train['쇼핑몰']=='S001-00010']
common = list(set(shop10['제품'].tolist())-(set(shop10['제품'].tolist())-set(shop4['제품'].tolist())))
for i in tqdm(common):
    a = pd.DataFrame(np.array(shop10[shop10['제품']==i].iloc[:,7:]) + np.array(shop4[shop4['제품']==i].iloc[:,7:]))
    k1 = pd.concat([shop10[shop10['제품']==i].iloc[:,:7].reset_index(drop=True),a],axis=1,ignore_index=True)
    k2 = pd.concat([shop4[shop4['제품']==i].iloc[:,:7].reset_index(drop=True),a],axis=1,ignore_index=True)
    shop10.loc[shop10['제품']==i] = list(np.array(k1))
    shop4.loc[shop4['제품']==i] = list(np.array(k2))

train[train['쇼핑몰']=='S001-00004'] = shop4
train[train['쇼핑몰']=='S001-00010'] = shop10

train.to_csv("./train2.csv",index=False) 

## ID별 일일판매액 전처리

In [ ]:
sales = pd.read_csv("./sales.csv")

# ['제품', '쇼핑몰'] 기준으로 groupby
grouped_data2 = sales.groupby(['제품','쇼핑몰'])
grouped_dfs2 = [group_df for _, group_df in grouped_data2]

# 동일한 쇼핑몰에서 판매하는 동일 제품들의 group index 찾음.
list2 = []
for i in tqdm(range(28863)):
    if len(grouped_dfs2[i]) != 1:
        list2.append(i)
        
# 동일한 쇼핑몰에서 판매하는 동일 제품들의 'ID'들끼리 일일판매액을 합친 후, 해당 'ID'의 일일판매액으로 대체함.
for i in tqdm(list2):
    a = pd.DataFrame(np.array(grouped_dfs2[i].iloc[0,7:]) + np.array(grouped_dfs2[i].iloc[1,7:]))
    k1 = pd.concat([grouped_dfs2[i].iloc[0,:7].reset_index(drop=True),a],axis=0,ignore_index=True)
    k2 = pd.concat([grouped_dfs2[i].iloc[1,:7].reset_index(drop=True),a],axis=0,ignore_index=True)
    sales.loc[train['ID']==k1.loc[0].tolist()[0]] = list(np.array(k1).reshape(-1,486))
    sales.loc[train['ID']==k2.loc[0].tolist()[0]] = list(np.array(k2).reshape(-1,486))

sales.to_csv("./sales1.csv",index=False)

# 쇼핑몰 4와 10에서 판매한 공통 제품들의 일일판매액을 합친 후, 해당 '제품'의 일일판매액으로 대체함. 
sales1 = pd.read_csv("./sales1.csv")
shop4 = sales1[sales1['쇼핑몰']=='S001-00004']
shop10 = sales1[sales1['쇼핑몰']=='S001-00010']
common = list(set(shop10['제품'].tolist())-(set(shop10['제품'].tolist())-set(shop4['제품'].tolist())))
for i in tqdm(common):
    a = pd.DataFrame(np.array(shop10[shop10['제품']==i].iloc[:,7:]) + np.array(shop4[shop4['제품']==i].iloc[:,7:]))
    k1 = pd.concat([shop10[shop10['제품']==i].iloc[:,:7].reset_index(drop=True),a],axis=1,ignore_index=True)
    k2 = pd.concat([shop4[shop4['제품']==i].iloc[:,:7].reset_index(drop=True),a],axis=1,ignore_index=True)
    shop10.loc[shop10['제품']==i] = list(np.array(k1))
    shop4.loc[shop4['제품']==i] = list(np.array(k2))

sales1[sales1['쇼핑몰']=='S001-00004'] = shop4
sales1[sales1['쇼핑몰']=='S001-00010'] = shop10

sales1.to_csv("./sales2.csv",index=False)

## 대/중/소분류/쇼핑몰 전처리

In [ ]:
df = pd.read_csv("./train1.csv") 
df999=df.copy()
df1=df.groupby("대분류").sum()
df2= df1.drop(["ID",'제품','중분류','소분류','브랜드','쇼핑몰'],axis=1)
df3= df2.transpose()

# 상관계수 행렬 계산
correlation_matrix = df3.corr()

# 거리 계산
distance_matrix = 1 - correlation_matrix

# 다차원 척도법(MDS)를 사용하여 1차원으로 축소
mds = MDS(dissimilarity="precomputed", n_components=1,random_state=1)
brand_positions = mds.fit_transform(distance_matrix)

df2["MDS"]=brand_positions
df2=df2.reset_index()
df21=df2.iloc[:,[0,-1]]

# 대분류 -> 대분류 MDS
df99=pd.merge(df,df21,on="대분류")
df99=df99.sort_values("ID").reset_index(drop=True)
df999['대분류']=df99["MDS"]

# 중분류
df1_1=df.groupby(["대분류","중분류"]).sum()
df2_1=df1_1.drop(["ID",'제품','소분류','브랜드','쇼핑몰'],axis=1)
df3_1= df2_1.transpose()

# 상관계수 행렬 계산
correlation_matrix_1 = df3_1.corr()

# 거리 계산
distance_matrix1 = 1 - correlation_matrix_1

# 다차원 척도법(MDS)를 사용하여 1차원으로 축소
brand_positions1 = mds.fit_transform(distance_matrix1)

df2_1["MDS"]=brand_positions1
df2_1=df2_1.reset_index()
df21_1=df2_1.iloc[:,[0,1,-1]]

# 중분류 -> 중분류 MDS
df99_1=pd.merge(df,df21_1,on=["대분류","중분류"])
df99_1=df99_1.sort_values("ID").reset_index(drop=True)
df999['중분류']=df99_1["MDS"]

# 소분류
df1_2=df.groupby("소분류").sum()
df2_2=df1_2.drop(["ID",'제품','대분류','중분류','브랜드','쇼핑몰'],axis=1)
df3_2= df2_2.transpose()

# 상관계수 행렬 계산
correlation_matrix_2 = df3_2.corr()

distance_matrix2 = 1 - correlation_matrix_2

# 다차원 척도법(MDS)를 사용하여 1차원으로 축소
brand_positions2 = mds.fit_transform(distance_matrix2)

df2_2["MDS"]=brand_positions2
df2_2=df2_2.reset_index()
df21_2=df2_2.iloc[:,[0,-1]]

# 소분류 -> 소분류 MDS
df99_2=pd.merge(df,df21_2,on="소분류")
df99_2=df99_2.sort_values("ID").reset_index(drop=True)
df999['소분류']=df99_2["MDS"]


#쇼핑몰
df1_3=df.groupby("쇼핑몰").sum()
df2_3=df1_3.drop(["ID",'제품','대분류','중분류','소분류','브랜드'],axis=1)
df3_3= df2_3.transpose()

# 상관계수 행렬 계산
correlation_matrix_3 = df3_3.corr()

distance_matrix3 = 1 - correlation_matrix_3

# 다차원 척도법(MDS)를 사용하여 1차원으로 축소
brand_positions3 = mds.fit_transform(distance_matrix3)

df2_3["MDS"]=brand_positions3
df2_3=df2_3.reset_index()
df21_3=df2_3.iloc[:,[0,-1]]

# 소분류 -> 소분류 MDS
df99_3=pd.merge(df,df21_3,on="쇼핑몰")
df99_3=df99_3.sort_values("ID").reset_index(drop=True)
df999['쇼핑몰']=df99_3["MDS"]

df999.to_csv('./train_MDS_train1ver.csv',index=False)

## 일일 개당 판매가 전처리

In [ ]:
sales = pd.read_csv("./sales2.csv")
del sales['대분류']
del sales['중분류']
del sales['소분류']
del sales['쇼핑몰']

brand_name = sales['브랜드'].tolist()
product_name = sales['제품'].tolist()
id_name = sales['ID'].tolist()
del sales['ID']
del sales['제품']
del sales['브랜드']

sales = sales.transpose()

full_data = []

for product_i in tqdm(range(len(id_name))):
    y = sales[product_i].tolist()
    data = [[i,y[i],product_name[product_i],brand_name[product_i],id_name[product_i]] for i in range(len(y))]
    full_data += data
price = pd.DataFrame(full_data, columns=['x','y','product','brand','ID'])

train = pd.read_csv("./train2.csv")
del train['대분류']
del train['중분류']
del train['소분류']
del train['쇼핑몰']

brand_name = train['브랜드'].tolist()
product_name = train['제품'].tolist()
id_name = train['ID'].tolist()
del train['제품']
del train['브랜드']
del train['ID']

train = train.transpose()

full_data = []

for product_i in tqdm(range(len(id_name))):
    y = train[product_i].tolist()
    data = [[i,y[i],product_name[product_i],brand_name[product_i],id_name[product_i]] for i in range(len(y))]
    full_data += data
sales_num_df = pd.DataFrame(full_data, columns=['x','y','product','brand','ID'])

def pre(x):
    if x == 0:
        return 1
    else:
        return x

sales_num = sales_num_df['y'].apply(pre)
def post(x):
    if x == 0:
        return np.nan
    else:
        return x

# 일일 개당 판매가 = 일일 총판매액/일일 총판매량
price['y1'] = price['y']/sales_num
price['y1'] = price['y1'].apply(post)

# 결측치 처리 : forward fill --> backward fill
for i in tqdm(range(28894)):
    price.iloc[i*479:(i+1)*479,5] = price.iloc[i*479:(i+1)*479,5].fillna(method='ffill')
    price.iloc[i*479:(i+1)*479,5] = price.iloc[i*479:(i+1)*479,5].fillna(method='bfill')


# 'ID'별 판매가율 = 일일 개당 판매가/해당 'ID'의 최대 개당 판매가
div_price = pd.DataFrame()
div_list = []
def s2i(x):
    return int(x.split("_")[1])
price1 = price.copy()
price1['ID'] = price1['ID'].apply(s2i)


div_price = pd.DataFrame()
div_list = []
def s2i(x):
    return int(x.split("_")[1])
price1 = price.copy()
price1['ID'] = price1['ID'].apply(s2i)


div_price = pd.DataFrame()
for i in tqdm(range(28894)):
    original_price = price1[price1['ID']==i]['y1']
    max_price = original_price.max()
    div = pd.DataFrame((np.array(original_price)/(max_price)).tolist())
    div_price = pd.concat([div_price,div],ignore_index=True)
    
price['y1'] = div_price

price.to_csv("./price.csv", index=False)

## 브랜드 언급량 전처리

In [ ]:
#브랜드 키워드 언급량 파일 불러오기
df = pd.read_csv("./brand_keyword_cnt.csv")

# 날짜 열들만 선택
date_columns = df.columns[1:]

# 날짜별로 median 값을 계산
median_values = df[date_columns].median()

# 모든 날짜에 대해 median 미만의 값을 가지는 브랜드명을 추출하고 그 브랜드의 값은 전부 0으로 변경
selected_brands = []
for brand in df['브랜드']:
    brand_values = df[df['브랜드'] == brand][date_columns].values
    if all(brand_values[0] < median_values):
        df.loc[df['브랜드'] == brand, date_columns] = 0
    else:
        # 나머지는 MinMaxScaling을 위한 브랜드 데이터 추출
        brand_data = df[df['브랜드'] == brand][date_columns]       
        # MinMaxScaler를 초기화하고 변환
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(np.array(brand_data).reshape(-1,1)).reshape(1,-1)
        # 변환된 값을 원본 데이터프레임에 업데이트
        df.loc[df['브랜드'] == brand, date_columns] = scaled_data

#데이터 병향을 위해 판매량 데이터 불러오기
df2=pd.read_csv("./train2.csv")

#각 제품의 브랜드에 맞게 브랜드언급량 scaling값 대입
df2_1=df2.iloc[:,:7]
df3=pd.merge(df2_1,df,on="브랜드")

#결측치 0값 대체
df3_1=df3.fillna(0)

#csv파일 저장
df3_1.to_csv("./HOT.csv",index=False)

## 판매량 0 전처리

In [ ]:
train = pd.read_csv("./train2.csv")

# 판매량이 없으면 1로 labeling, 있으면 0으로 labeling
def isnull(x):
    if x==0:
        return 1
    else:
        return 0

df1 = train.iloc[:,7:]
df1 = df1.transpose()
for k in tqdm(range(len(df1.columns))):
    df1.iloc[:,k]=df1.iloc[:,k].apply(isnull) 

null_index = df1.transpose()
null_index.to_csv("./null_index.csv",index=False)

# 소분류별 시장 동향 예측 특성 추가

## XGBoost train dataset : 날짜, 소분류 encoding 및 판매 지수 labeling

In [ ]:
# 2022년 XGB train dataset
train = pd.read_csv("./train2.csv")

data = train.iloc[:,7:].columns
date_df = pd.DataFrame(data)
date_df['Date'] = pd.to_datetime(date_df.iloc[:,0])
date_df['Year'] = date_df['Date'].dt.year
date_df['Month'] = date_df['Date'].dt.month
date_df['Day'] = date_df['Date'].dt.day

# 월/일 encoding
date_365 = date_df.iloc[:365,3:5] 
date_365_df = date_365.copy()
for i in tqdm(range(28894-1)):
    date_365_df = pd.concat([date_365_df,date_365],ignore_index=True) 

# 요일 encoding
week = ([6,7,1,2,3,4,5]*52+[6])*28894 
date_365_df['Week'] = pd.DataFrame(week) 

holiday_list1 = ['2022-01-01','2022-03-01','2022-03-09','2022-05-05','2022-05-08',
                '2022-06-01','2022-06-06','2022-08-15','2022-10-03',
                '2022-10-09','2022-10-10','2022-12-25']

holiday_list2 = ['2022-01-29','2022-01-30','2022-01-31','2022-02-01','2022-02-02',
                 '2022-09-09','2022-09-10','2022-09-11','2022-09-12']

# 공휴일 1로 labeling
selected_rows1 = pd.DataFrame() 
for i in holiday_list1:
    selected_rows1 = pd.concat([selected_rows1,date_df[date_df[0]==i]])
    selected_rows1['Holiday'] = 1

# 설날/추석 연휴 2로 labeling
selected_rows2 = pd.DataFrame()
for i in holiday_list2:
    selected_rows2 = pd.concat([selected_rows2,date_df[date_df[0]==i]])
    selected_rows2['Holiday'] = 2

selected_rows = pd.concat([selected_rows1, selected_rows2],ignore_index=True)
selected_rows = selected_rows.iloc[:,3:6]
result = pd.merge(date_365_df, selected_rows,how='left')

# 평일 0으로 labeling
result['Holiday'] = result['Holiday'].fillna(0)

# 소분류 encoding
k = train['소분류']
for i in range(364):
    k = pd.concat([k,train['소분류']],axis=1)
k = k.values.flatten()
k = pd.DataFrame(k)
train_XGB_df = pd.concat([result,k],ignore_index=True,axis=1) 

train_XGB_df = train_XGB_df.rename(columns={0:'Month',1:'Day',2:'Week',3:'Holiday',4:'소분류'})

def split_sub(x):
    return int(x.split("-")[2])    
train_XGB_df['소분류'] = train_XGB_df['소분류'].apply(split_sub)

# 판매 지수 labeling
train_365 = train.iloc[:,7:372]
for i in tqdm(range(28894)):
    top = train_365.iloc[i,:].quantile(0.80)
    bot = train_365.iloc[i,:].quantile(0.20)
    for k in range(365):
        x = train_365.iloc[i,k]
        if x > top:
            train_365.iloc[i,k] = 1
        elif x <= bot:
            train_365.iloc[i,k] = 2
        else:
            train_365.iloc[i,k] = 0

train_XGB_df['y'] = pd.DataFrame(train_365.values.flatten())

train_XGB_df.to_csv("./train_XGB_df.csv",index=False)

## XGBoost inference dataset : 날짜, 소분류 encoding

In [ ]:
# 2023/4/24까지 XGB infer dataset
train = pd.read_csv("./train2.csv")

data = train.iloc[:,7:].columns
date_df = pd.DataFrame(data)
date_df['Date'] = pd.to_datetime(date_df.iloc[:,0])
date_df['Year'] = date_df['Date'].dt.year
date_df['Month'] = date_df['Date'].dt.month
date_df['Day'] = date_df['Date'].dt.day

# 월/일 encoding
date_479 = date_df.iloc[:479,3:5] 
date_479_df = date_479.copy()
for i in tqdm(range(28894-1)):
    date_479_df = pd.concat([date_479_df,date_479],ignore_index=True)

# 요일 encoding
week = ([6,7,1,2,3,4,5]*68+[6,7,1])*28894 
date_479_df['Week'] = pd.DataFrame(week) 

holiday_list1 = ['2022-01-01','2022-03-01','2022-03-09','2022-05-05','2022-05-08',
                '2022-06-01','2022-06-06','2022-08-15','2022-10-03','2022-10-09',
                '2022-10-10','2022-12-25','2023-01-01','2023-03-01']
holiday_list2 = ['2022-01-29','2022-01-30','2022-01-31','2022-02-01','2022-02-02',
                 '2022-09-09','2022-09-10','2022-09-11','2022-09-12','2023-01-21',
                 '2023-01-22','2023-01-23','2023-01-24']

# 공휴일 1로 labeling
selected_rows1 = pd.DataFrame() 
for i in holiday_list1:
    selected_rows1 = pd.concat([selected_rows1,date_df[date_df[0]==i]])
    selected_rows1['Holiday'] = 1

# 설날/추석 연휴 2로 labeling
selected_rows2 = pd.DataFrame()
for i in holiday_list2:
     selected_rows2 = pd.concat([selected_rows2,date_df[date_df[0]==i]])
     selected_rows2['Holiday'] = 2

selected_rows = pd.concat([selected_rows1, selected_rows2],ignore_index=True)
selected_rows.drop(columns=['Date','Year','Month','Day'],inplace=True)
resultt = pd.merge(date_df,selected_rows,on=0,how='left')
resultt = resultt.iloc[:,3:]

# 평일 0으로 labeling
resultt['Holiday'] = resultt['Holiday'].fillna(0)

resultt = resultt['Holiday']
resulttt = resultt.copy()
for k in tqdm(range(28894-1)):
    resultt = pd.concat([resultt,resulttt],ignore_index=True)

date_479_df['Holiday'] = resultt

# 소분류 encoding
m = train['소분류']
for i in tqdm(range(479-1)):
    m = pd.concat([m,train['소분류']],axis=1)
m = m.values.flatten()
m = pd.DataFrame(m)
infer_XGB_479_df = pd.concat([date_479_df,m],ignore_index=True,axis=1) 

infer_XGB_479_df = infer_XGB_479_df.rename(columns={0:'Month',1:'Day',2:'Week',3:'Holiday',4:'소분류'})

def split_sub(x):
    return int(x.split("-")[2])    
infer_XGB_479_df['소분류'] = infer_XGB_479_df['소분류'].apply(split_sub)

infer_XGB_479_df.to_csv("./infer_XGB_479_df.csv",index=False)

In [ ]:
# 2023/5/15까지 XGB infer dataset
train = pd.read_csv("./train2.csv")

data = train.iloc[:,7:].columns
date_df = pd.DataFrame(data)
date_df['Date'] = pd.to_datetime(date_df.iloc[:,0])
date_df['Year'] = date_df['Date'].dt.year
date_df['Month'] = date_df['Date'].dt.month
date_df['Day'] = date_df['Date'].dt.day

# 월/일 encoding
date_479 = date_df.iloc[:479,3:5] 
date_500 = pd.concat([date_479,date_479.iloc[114:135,:]],ignore_index=True) 
date_500_df = date_500.copy() 
for i in tqdm(range(28894-1)):
    date_500_df = pd.concat([date_500_df,date_500],ignore_index=True)

# 요일 encoding
week = ([6,7,1,2,3,4,5]*71+[6,7,1])*28894 
date_500_df['Week'] = pd.DataFrame(week)

# 공휴일 labeling
infer_XGB_500 = infer_XGB_479_df.iloc[:479,2]
result = pd.DataFrame(np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]))
infer_500_holiday = pd.concat([infer_XGB_500,result])
infer_500_holiday_data = infer_500_holiday.copy()
for i in tqdm(range(28894-1)):
    infer_500_holiday = pd.concat([infer_500_holiday,infer_500_holiday_data],ignore_index=True)

date_500_df['Holiday'] = infer_500_holiday

# 소분류 encoding
m = train['소분류']
for i in range(500-1):
    m = pd.concat([m,train['소분류']],axis=1)
m = m.values.flatten()
m = pd.DataFrame(m)
infer_XGB_500_df = pd.concat([date_500_df,m],ignore_index=True,axis=1) 

infer_XGB_500_df = infer_XGB_500_df.rename(columns={0:'Month',1:'Day',2:'Week',3:'Holiday',4:'소분류'})

def split_sub(x):
    return int(x.split("-")[2])    
infer_XGB_500_df['소분류'] = infer_XGB_500_df['소분류'].apply(split_sub)

infer_XGB_500_df.to_csv("./infer_XGB_500_df.csv",index=False)

## XGBoost train dataset을 이용하여 소분류별 XGBoost model 학습 및 추론

In [ ]:
train = pd.read_csv("./train2.csv")

# 소분류 기준으로 groupby
train_XGB = train_XGB_df.groupby('소분류')
train_XGB_gb_df = [group_df for _, group_df in train_XGB]

infer_XGB_479 = infer_XGB_479_df.groupby('소분류')
infer_XGB_479_gb_df = [group_df for _, group_df in infer_XGB_479]

infer_XGB_500 = infer_XGB_500_df.groupby('소분류')
infer_XGB_500_gb_df = [group_df for _, group_df in infer_XGB_500]

proba_XGB_479_df = pd.DataFrame()
proba_XGB_500_df = pd.DataFrame()

for i in tqdm(range(53)):
    # up-sampling
    X1 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==2].iloc[:,:-2]
    X2 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==0].iloc[:,:-2]
    X3 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==1].iloc[:,:-2]
    y1 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==2].iloc[:,-1]
    y2 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==0].iloc[:,-1]
    y3 = train_XGB_gb_df[i][train_XGB_gb_df[i]['y']==1].iloc[:,-1]
    
    if train_XGB_gb_df[i]['y'].value_counts()[2] == 0:
        vm = 0
    else:
        vm = round(train_XGB_gb_df[i]['y'].value_counts().max() / train_XGB_gb_df[i]['y'].value_counts()[2])

    if train_XGB_gb_df[i]['y'].value_counts()[0] == 0:
        vz = 0
    else:
        vz = round(train_XGB_gb_df[i]['y'].value_counts().max() / train_XGB_gb_df[i]['y'].value_counts()[0])
    
    if train_XGB_gb_df[i]['y'].value_counts()[1] == 0:
        vo = 0
    else:
        vo = round(train_XGB_gb_df[i]['y'].value_counts().max() / train_XGB_gb_df[i]['y'].value_counts()[1])
    
    if vm == 1:
        X = X1
        y = y1
        for j in range(vz):
            X = pd.concat([X,X2])
            y = pd.concat([y,y2])
        for k in range(vo):
            X = pd.concat([X,X3])
            y = pd.concat([y,y3])
    elif vz == 1:
        X = X2
        y = y2
        for j in range(vm):
            X = pd.concat([X,X1])
            y = pd.concat([y,y1])
        for k in range(vo):
            X = pd.concat([X,X3])
            y = pd.concat([y,y3])
    elif vo == 1:
        X = X3
        y = y3
        for j in range(vm):
            X = pd.concat([X,X1])
            y = pd.concat([y,y1])
        for k in range(vz):
            X = pd.concat([X,X2])
            y = pd.concat([y,y2])

    # 소분류별 XGBoost model train    
    clf = xgb.XGBClassifier(random_state=0)
    clf.fit(X,y)
    
    # 소분류별 XGBoost model inference(2023/4/24까지)
    proba_list = clf.predict_proba(infer_XGB_479_gb_df[i].iloc[:479,:-1])
    pl_df = pd.DataFrame(proba_list, columns=['0 proba', '1 proba', '-1 proba'])
    
    pl_df['peak_index'] = pl_df['1 proba']-pl_df['-1 proba']
    del pl_df['0 proba']
    del pl_df['1 proba']
    del pl_df['-1 proba']
    pl_df = pl_df.transpose()
    
    if i < 9:
        pl_df['소분류'] =  f"B002-C003-000{i+1}"
    else:
        pl_df['소분류'] =  f"B002-C003-00{i+1}"
    proba_XGB_479_df = pd.concat([proba_XGB_479_df,pl_df])

    # 소분류별 XGBoost model inference(2023/5/15까지)
    proba_listt = clf.predict_proba(infer_XGB_500_gb_df[i].iloc[:500,:-1])
    pll_df = pd.DataFrame(proba_listt, columns=['0 proba', '1 proba', '-1 proba'])
    
    pll_df['peak_index'] = pll_df['1 proba']-pll_df['-1 proba']
    del pll_df['0 proba']
    del pll_df['1 proba']
    del pll_df['-1 proba']
    pll_df = pll_df.transpose()
    
    if i < 9:
        pll_df['소분류'] =  f"B002-C003-000{i+1}"
    else:
        pll_df['소분류'] =  f"B002-C003-00{i+1}"
    proba_XGB_500_df = pd.concat([proba_XGB_500_df,pll_df])

pi = pd.merge(train['소분류'],proba_XGB_479_df,how='left',on='소분류')
peak_424 = pd.DataFrame(pi.iloc[:,1:].values.flatten())
peak_424.to_csv("./peak_index.csv",index=False)

pii = pd.merge(train['소분류'],proba_XGB_500_df,how='left',on='소분류')
peak_515 = pd.DataFrame(pii.iloc[:,1:].values.flatten())
peak_515.to_csv("./peak_index_500.csv",index=False)

# Train 환경설정

In [ ]:
# GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'TRAIN_WINDOW_SIZE':90, 
    'PREDICT_SIZE':21, 
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':1024,
    'SEED':41
}

# Seed값 고정 함수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# 대분류별 train/inference dataset 생성

In [ ]:
def split_sub(x):
    return int(x.split("_")[1])  

temp = pd.read_csv('./train2.csv')
temp['ID'] = temp['ID'].apply(split_sub)
temp.to_csv('./train2.csv', index=False)

temp = pd.read_csv('./sales2.csv')
temp['ID'] = temp['ID'].apply(split_sub)
temp.to_csv('./sales2.csv', index=False)

temp = pd.read_csv('./HOT.csv')
temp['ID'] = temp['ID'].apply(split_sub)
temp.to_csv('./HOT.csv', index=False)

temp = pd.read_csv('./price.csv')
temp['ID'] = temp['ID'].apply(split_sub)
temp.to_csv('./price.csv', index=False)

temp = pd.read_csv('./train_MDS_train1ver.csv')
temp['ID'] = temp['ID'].apply(split_sub)
temp.to_csv('./train_MDS_train1ver.csv', index=False)

In [ ]:
train_data = pd.read_csv('./train2.csv').drop(columns=['제품', 'ID','브랜드'])
train_data1 = pd.read_csv("./train_MDS_train1ver.csv")

n_bigcat_list = train_data.value_counts('대분류').sort_index().to_list()
index1 = (train_data['대분류']=='B002-C001-0001')
index2 = (train_data['대분류']=='B002-C001-0002')
index3 = (train_data['대분류']=='B002-C001-0003')
index4 = (train_data['대분류']=='B002-C001-0004')
index5 = (train_data['대분류']=='B002-C001-0005')
train_data['대분류'] = train_data1['대분류']
train_data['중분류'] = train_data1['중분류']
train_data['소분류'] = train_data1['소분류']
train_data['쇼핑몰'] = train_data1['쇼핑몰']

In [ ]:
def make_data(train_data, index_list):
    df2 = pd.read_csv("./price.csv")
    del df2['x']
    del df2['y']
    del df2['product']
    del df2['brand']

    df3 = pd.read_csv("./HOT.csv").drop(columns=["ID","제품","대분류","중분류","소분류","브랜드","쇼핑몰"])
    df4 = pd.read_csv("./null_index.csv")
    df5 = pd.read_csv("./peak_index.csv")
    hot_list = []
    null_list = []
    price_list = []
    peak_list = []
    for j in tqdm(range(len(index_list[index_list==True].index))):
        exec(f"price_list += df2[df2['ID']==index_list[{j}]]['y1'].tolist()")
        exec(f"hot_list += df3.iloc[index_list[index_list==True].index[{j}],:].tolist()")
        exec(f"null_list += df4.iloc[index_list[index_list==True].index[{j}],:].tolist()")
        exec(f"peak_list += df5.iloc[index_list[index_list==True].index[{j}]*479:(index_list[index_list==True].index[{j}]+1)*479].to_numpy().reshape(-1).tolist()")

    return hot_list, peak_list, null_list, price_list

In [ ]:
def make_train_data(data, other_data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):

    hot_data, peak_data, null_data, price_data = other_data

    num_rows = len(data) 
    window_size = train_size + predict_size 
    input_data = np.empty((num_rows * (len(data.columns) - 4 - window_size + 1), train_size, 9))

    target_data = np.empty((num_rows * (len(data.columns) - 4 - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:]) 
        hot_data_i = np.array(hot_data[i*479:(i+1)*479])
        peak_data_i = np.array(peak_data[i*479:(i+1)*479])
        null_data_i = np.array(null_data[i*479:(i+1)*479])
        price_data_i = np.array(price_data[i*479:(i+1)*479])
        for j in range(len(data.columns) - 4 - window_size + 1): 
            window = sales_data[j : j + window_size]
            
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size])).tolist()
            
            temp_list2 = []
            for k in range(len(temp_data)):
                temp_list = temp_data[k]
                temp_list += [hot_data_i[j+k],peak_data_i[j+k],null_data_i[j+k],price_data_i[j+k]]
                
                temp_list2.append(temp_list)
            temp_data = temp_list2
            
            input_data[i * (len(data.columns) - 4 - window_size + 1) + j] = temp_data 
            target_data[i * (len(data.columns) - 4 - window_size + 1) + j] = window[train_size:] 

    return input_data, target_data

In [ ]:
# 대분류별 encoder input window dataset과 decoder output window dataset 생성
train_input1, train_target1 = make_train_data(train_data[index1], make_data(train_data, index1))
train_input2, train_target2 = make_train_data(train_data[index2], make_data(train_data, index2))
train_input3, train_target3 = make_train_data(train_data[index3], make_data(train_data, index3))
train_input4, train_target4 = make_train_data(train_data[index4], make_data(train_data, index4))
train_input5, train_target5 = make_train_data(train_data[index5], make_data(train_data, index5))

In [ ]:
def make_proba_data(data, other_data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
 
    hot_data, peak_data, null_data, price_data = other_data
    
    num_rows = len(data) 
    window_size = train_size + predict_size 
    target_data = np.empty((num_rows * (len(data.columns) - 4 - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)): 
        peak_data_i = np.array(peak_data[i*479:(i+1)*479])
        for j in range(len(data.columns) - 4 - window_size + 1): 
            window = peak_data_i[j : j + window_size]
            target_data[i * (len(data.columns) - 4 - window_size + 1) + j] = window[train_size:] 

    return target_data

In [ ]:
# 대분류별 decoder input window dataset 생성
proba_data1 = make_proba_data(train_data[index1], make_data(train_data, index1))
proba_data2 = make_proba_data(train_data[index2], make_data(train_data, index2))
proba_data3 = make_proba_data(train_data[index3], make_data(train_data, index3))
proba_data4 = make_proba_data(train_data[index4], make_data(train_data, index4))
proba_data5 = make_proba_data(train_data[index5], make_data(train_data, index5))

In [ ]:
class CustomDataset2(Dataset):
    def __init__(self, X):
        self.X = torch.tensor(X).type(torch.float32)

    def __getitem__(self, index):
        return torch.Tensor(self.X[index]).type(torch.float32)

    def __len__(self):
        return len(self.X)

In [ ]:
def make_predict_data(data, other_data, train_size=CFG['TRAIN_WINDOW_SIZE']):
  
    num_rows = len(data)
    hot_data, peak_data, null_data, price_data = other_data
  
    input_data = np.empty((num_rows, train_size, 9))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        hot_data_i = np.array(hot_data[(i+1)*479-train_size:(i+1)*479])
        peak_data_i = np.array(peak_data[(i+1)*479-train_size:(i+1)*479])
        null_data_i = np.array(null_data[(i+1)*479-train_size:(i+1)*479])
        price_data_i = np.array(price_data[(i+1)*479-train_size:(i+1)*479])

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size])).tolist()
        temp_list2 = []
        for k in range(len(temp_data)):
            temp_list = temp_data[k]
            temp_list += [hot_data_i[k],peak_data_i[k],null_data_i[k],price_data_i[k]]
            
            temp_list2.append(temp_list)
        temp_data = temp_list2
        input_data[i] = temp_data

    return input_data

In [ ]:
# encoder inference input window dataset 생성
test_input = make_predict_data(train_data, make_data(train_data, train_data['대분류'].notna()))

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X).type(torch.float32)
        self.Y = torch.tensor(Y).type(torch.float32)

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]).type(torch.float32), torch.Tensor(self.Y[index]).type(torch.float32)
        return torch.Tensor(self.X[index]).type(torch.float32)

    def __len__(self):
        return len(self.X)

# Model architecture

In [ ]:
# LSTM + Regressor 모델
class BaseModel(nn.Module):
    def __init__(self, input_size=9, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()
        self.re = nn.Sequential(
            nn.Linear(output_size*2, hidden_size),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

    def forward(self, x, x1):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        lstm_out, hidden = self.lstm(x, hidden)
        
        last_output = lstm_out[:, -1, :]

        output = self.actv(self.fc(last_output))
        output = output.squeeze(1)
        x1 = output * x1
        x2 = torch.cat((output,x1),dim=1)
        output = self.actv(self.re(x2))

        return output

    def init_hidden(self, batch_size, device):
       
        return (torch.zeros(1, batch_size, self.hidden_size, device=device).type(torch.float32),
                torch.zeros(1, batch_size, self.hidden_size, device=device).type(torch.float32))

# Objective function

In [ ]:
def PSFA_t(pred, target,device):

    PSFA = 1
    ids = range(len(target))
    for day in range(21): 
        total_sell = torch.tensor([]).to(device)
        pred_values = torch.tensor([]).to(device)
        target_values = torch.tensor([]).to(device)
        for index in ids: 
            pred_values=torch.cat([pred_values,pred[index][day].reshape(1)]).to(device) 
            target_values=torch.cat([target_values,target[index][day].reshape(1)]).to(device) 
        total_sell = torch.sum(target_values).to(device)  
        
        pred_values = torch.nan_to_num(pred_values).to(device)
        pred_values = torch.maximum(torch.zeros_like(pred_values),pred_values).to(device)
        denominator = torch.maximum(target_values, pred_values).to(device) 

        diffs = torch.where(denominator!=0, torch.abs(target_values - pred_values)/ denominator, 0).to(device)

        if total_sell != 0:
            sell_weights = target_values / total_sell  
        else:
            sell_weights = torch.zeros_like(target_values).to(device) 

        if not torch.isnan(diffs).any():  
            PSFA -= torch.sum(diffs * sell_weights).to(device) / (21)


    return PSFA

In [ ]:
def PSFA_v(pred, target):
    
    PSFA = 1
    ids = range(len(target))
    for day in range(21): 
        total_sell = np.array([])
        pred_values = np.array([])
        target_values = np.array([])
        for index in ids: 
            pred_values=np.concatenate([pred_values,pred[index][day].reshape(1)])
            target_values=np.concatenate([target_values,target[index][day].reshape(1)]) 
        total_sell = np.sum(target_values)  
        pred_values = np.nan_to_num(pred_values)
        pred_values = np.maximum(np.zeros_like(pred_values),pred_values)
        denominator = np.maximum(target_values, pred_values) 

        diffs = np.where(denominator!=0, np.abs(target_values - pred_values) / denominator, 0)

        if total_sell != 0:
            sell_weights = target_values / total_sell  
        else:
            sell_weights = np.zeros_like(target_values) 

        if not np.isnan(diffs).any():  
            PSFA -= np.sum(diffs * sell_weights) / (21) 


    return PSFA

# Validation

In [ ]:
def validation(model, n_bigcat_list, train_input1, train_target1, train_input2, train_target2, train_input3, train_target3, train_input4, train_target4, train_input5, train_target5, proba_data1, proba_data2, proba_data3, proba_data4, proba_data5):
    val_accuracy = 0
    for timeitem in [114, 368]:
        model.eval()
        batch_n_bigcat_list = (np.array(n_bigcat_list)).astype(int)
        batch_time_list = random.sample([timeitem], k = 1)
        for k in range(100):
            for i in range(1,6): 
                exec(f"batch_index_{i}_bigcat_{k} = (random.sample(range(n_bigcat_list[{i-1}]*{k}//100,n_bigcat_list[{i-1}]*(k+1)//100), k=len(range(n_bigcat_list[{i-1}]*{k}//100,n_bigcat_list[{i-1}]*(k+1)//100))))")

        for k in range(100):
            for i in range(1,6):

                batch_index = eval(f"batch_index_{i}_bigcat_{k}[0]")
                exec(f"batch_train_input{i}_{k} = train_input{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                exec(f"batch_train_target{i}_{k} = train_target{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                exec(f"batch_proba_data{i}_{k} = proba_data{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                for batch_index in eval(f"batch_index_{i}_bigcat_{k}[1:]"):
                    temp1 = eval(f"train_input{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    temp2 = eval(f"train_target{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    temp3 = eval(f"proba_data{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    exec(f"batch_train_input{i}_{k} = np.concatenate((batch_train_input{i}_{k}, temp1[batch_time_list]))")
                    exec(f"batch_train_target{i}_{k} = np.concatenate((batch_train_target{i}_{k}, temp2[batch_time_list]))")
                    exec(f"batch_proba_data{i}_{k} = np.concatenate((batch_proba_data{i}_{k}, temp3[batch_time_list]))")
                exec(f"train_dataset{i}_{k} = CustomDataset(batch_train_input{i}_{k}, batch_train_target{i}_{k})")
                exec(f"proba_dataset{i}_{k} = CustomDataset2(batch_proba_data{i}_{k})")

        total_accuracy = 0
        total_loss = 0
        for i in range(1,6): 
            pred = 0
            target = 0
            for k in range(100):
                X = eval(f"train_dataset{i}_{k}.X.to(device)")
                Y = eval(f"train_dataset{i}_{k}.Y.to(device)")
                X1 = eval(f"proba_dataset{i}_{k}.X.to(device)")
                output = model(X, X1)

                pred1 = torch.nan_to_num(output)
                pred1 = torch.maximum(torch.zeros_like(pred1),pred1)
                target1 = Y
                pred1 = pred1.detach().cpu().numpy()
                target1 = target1.detach().cpu().numpy()
                if k == 0:
                    pred = pred1
                    target = target1
                else:
                    pred = np.concatenate((pred,pred1))
                    target = np.concatenate((target,target1))
            total_accuracy += PSFA_v(pred, target)
        val_accuracy += (total_accuracy)/5
    val_accuracy /= 2
    return val_accuracy 

# Train

In [ ]:
def train(model, optimizer, n_bigcat_list, train_input1, train_target1, train_input2, train_target2, train_input3, train_target3, train_input4, train_target4, train_input5, train_target5, proba_data1, proba_data2, proba_data3, proba_data4, proba_data5, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_accuracy = 0
    best_loss = 1000
    best_model = None
    accuracy_list = []

    # train 시, 이상치 제거
    for i in range(1,6):
        df = pd.read_csv("./train2.csv")[pd.read_csv("./train2.csv")['대분류']==f'B002-C001-000{i}'].reset_index(drop=True)
        df1 = df.iloc[:,7:].replace(0,np.nan)

        exec(f"not_in{i}=set(list(df1.isna().sum(axis=1)[df1.isna().sum(axis=1)>=420].index))")

    for epoch in range(1, 71):

        model.train()
        train_accuracy = []
        train_loss = []
        train_mae = []

        # 대분류별 10% batch로 뽑음
        id_ratio = 0.1
        for inner_epoch in tqdm(range(30)):

            batch_n_bigcat_list = (np.array(n_bigcat_list) * id_ratio+1).astype(int)
            
            # 시점 sampling
            if inner_epoch % 3 == 0:
                batch_time_list = random.sample(list(range(114))+list(range(115,123)), k = 1)
            elif inner_epoch % 3 == 1:
                batch_time_list = random.sample(range(123,246), k = 1)
            else:
                batch_time_list = random.sample(range(246,368), k = 1)

            for i in range(1,6):
                exec(f"batch_index_{i}_bigcat = (random.sample(list(set(range(n_bigcat_list[{i-1}]))-not_in{i}), k=min(batch_n_bigcat_list[{i-1}], len(list(set(range(n_bigcat_list[{i-1}]))-not_in{i})))))")

            for i in range(1,6):
                batch_index = eval(f"batch_index_{i}_bigcat[0]")
                exec(f"batch_train_input{i} = train_input{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                exec(f"batch_train_target{i} = train_target{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                exec(f"batch_proba_data{i} = proba_data{i}[{batch_index*369}:{(batch_index+1)*369}][batch_time_list]")
                for batch_index in eval(f"batch_index_{i}_bigcat[1:]"):
                    temp1 = eval(f"train_input{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    temp2 = eval(f"train_target{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    temp3 = eval(f"proba_data{i}[{batch_index*369}:{(batch_index+1)*369}]")
                    exec(f"batch_train_input{i} = np.concatenate((batch_train_input{i}, temp1[batch_time_list]))")
                    exec(f"batch_train_target{i} = np.concatenate((batch_train_target{i}, temp2[batch_time_list]))")
                    exec(f"batch_proba_data{i} = np.concatenate((batch_proba_data{i}, temp3[batch_time_list]))")

                exec(f"train_dataset{i} = CustomDataset(batch_train_input{i}, batch_train_target{i})")
                exec(f"proba_dataset{i} = CustomDataset2(batch_proba_data{i})")

            optimizer.zero_grad()
            total_accuracy = 0
            total_loss = 0
            for i in range(1,6): 
                # X : encoder input window 
                X = eval(f"train_dataset{i}.X.to(device)")
                # Y : decoder output window
                Y = eval(f"train_dataset{i}.Y.to(device)")
                # X1 : decoder input window
                X1 = eval(f"proba_dataset{i}.X.to(device)")
                output = model(X, X1).to(device)

                pred = torch.nan_to_num(output).to(device)
                pred = torch.maximum(torch.zeros_like(pred),pred).to(device)
                target = Y.to(device)
                total_accuracy += PSFA_t(pred, target,device).to(device)

            total_accuracy /= 5
            (-total_accuracy).backward()

            optimizer.step()

            train_accuracy.append(total_accuracy.item())

        val_accuracy = validation(model, n_bigcat_list, train_input1, train_target1, train_input2, train_target2, train_input3, train_target3, train_input4, train_target4, train_input5, train_target5, proba_data1, proba_data2, proba_data3, proba_data4, proba_data5)

        print(f'Epoch : [{epoch}] Train Accuracy : [{np.mean(train_accuracy):.5f}] Val Accuracy : [{val_accuracy:.5f}]')
        torch.save(model, f"./model{epoch}.pt")
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model = model
            torch.save(model, "./best_val_model.pt")

    return best_model

In [ ]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, n_bigcat_list, train_input1, train_target1, train_input2, train_target2, train_input3, train_target3, train_input4, train_target4, train_input5, train_target5, proba_data1, proba_data2, proba_data3, proba_data4, proba_data5, device)

# Inference

In [ ]:
class CustomDataset1(Dataset):
    def __init__(self, X):
        self.X = torch.tensor(X).type(torch.float32)

    def __getitem__(self, index):
        return torch.Tensor(self.X[index]).type(torch.float32)

    def __len__(self):
        return len(self.X)

In [ ]:
test_dataset = CustomDataset1(test_input)

In [ ]:
def inference(model, X, X1, device):
    predictions = []

    model.eval()

    output = model(X, X1)

    output = output.detach().cpu().numpy()
    predictions.extend(output)

    return np.array(predictions)

In [ ]:
infer_model = torch.load("./best_val_model.pt")
infer_model.eval()

for i in range(10):
    slice = len(test_dataset.X)//10
    X = test_dataset.X.to(device)[i*slice:(i+1)*slice]
    X1 = torch.tensor(pd.DataFrame(pd.read_csv("./peak_index_500.csv").to_numpy().reshape(-1,500)).iloc[:,-21:].to_numpy()).type(torch.float32).to(device)[i*slice:(i+1)*slice]
    exec(f"pred{i} = inference(infer_model, X, X1, device)")
    exec(f"pred{i} = np.maximum(np.zeros_like(pred{i}),pred{i})")
    exec(f"pred{i} = np.round(pred{i}, 0).astype(int)")

submit = pd.read_csv('./sample_submission.csv')
slice = len(test_dataset.X)//10
for i in range(10):
    exec(f"submit.iloc[i*slice:(i+1)*slice,1:] = pred{i}")
submit.to_csv('./submit_file.csv', index=False)